## Authentication

Authentication for this notebook will be via IAM Identity Center (Formerly SSO). Once you have logged into SSO for this notebook, all AWS actions will take place under your SSO credentials. You will need access across the organization to run the assessment.

In [1]:
import boto3, os
import botocore.exceptions
from jupyterirtools import sso
import ransomware

org_root_account = "383086473915"
configuration_aggregator = 'aggregator-dr1zj3ositog'
active_regions = [
    "us-east-1",  "us-east-2",  "us-west-1",  "us-west-2"
]
role = "Jupyter-IR-AdministratorAccess"
destination_bucket = "gillemi-gillemi"
destination_prefix = "jupyter-reports/"
export_to_security_hub = False

sso.login(role, org_root_account)



Current cached SSO login is expired or invalid
Fetching credentials again


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


If the login window doesn't automatically open, click to [activate the session](https://device.sso.us-east-1.amazonaws.com/?user_code=WFKV-WNFD)

<IPython.core.display.Javascript object>

Waiting for login...
Waiting for login...


Credentials expire in 7 hours and 59 minutes

Login Successful


Login Successful, click to open [AWS Console](https://signin.aws.amazon.com/federation?Action=login&Issuer=Example.org&Destination=https%3A%2F%2Fconsole.aws.amazon.com%2F&SigninToken=nBBSkWhKEuiJV1iYfUmV4nTzUm7uER-3AVGOzo1sqIJn0k91elQtHAC37zoyeKmalcJxpUKkcIrZeWkywUxxkqo4PYvrQYKbD-C9jnMLOrnA7-OkDYV5y-6z2eVbJK38sYuwT2P5_v0BJgYEjMCMweJjlbQtzRbLFO1RrmHKXZ-7JSkYTQp5HVdSl3D6QPZOv0Df5C9fu85sk9KZ1y45vwIVI3OMYHnrwmTcz3gZlIGkmpHO7sOfaubAt2sJRH-rZeresL2ruqXF_3Pc5nmN79FAoft9-UHiP45VP9WO9AVbwE8V2fA3PzHd42adL2w4CU9YNGxV8sXTmgCgSqYsR8n2VEStIUA47o1qniMA8gdYNQjYpeBu8ZVhxQIm-MtaQN14IvEX7gL67V6FwyyUjEU9OS8e4V4MRLEuDovMKeA7lZfocdixmO20hl9hGiPq9kcpd4gFXenvPW74OMoxLOb4zMFv8afikjtBgDeE6DgmiFnD1jTz6sBZFKxm0DgYHnEdHzyOkSXy5iaBDXjO5fRxfJ8RjTG4hHU67xjpJ57BZVxYMF6T2qHAVRj5RYpWQzvj631xvyhm7CODxlAhF-9GCJV-9NG45nSPyC2VZZj4nNj_S1YSZwWClGvgRoV2TqcN_7llge_8LwV5p6crf_H9FSKVxbmwr8alFr9R96oaY9Q-DpxegMgmnA6GRxgI4fTg69o3xeJ4ug6sQ9rPLCflOxWgt6_h0Y55YLXrcc8MTc-It06HiCXiFlw1XS6VyLnRnr6xbJ27IIAgbfUs7GrcwMeJjg0vqJzAWFFMxr7PXOy_Yiluud6giB2qTw2AVcZQbkqZ6iQJ0p8fjQrFOjMDb5vgXoiqqE8QwWauV_LkZh04J_KiWCZTgck2dk9h7gyhtG5IX1ICpbRUDH1lrv92yaHHYjLP-jIJ9jpJmQ9QZLsBQ0Q1hzTA9Hx2RtIDXgTVmDUTMXk_s6IgiWZIrjHJ8kplFLU6uV9b-GjbZt0H8FpoXMVpz0XeOAme5M1YPjqhMBHVu9jAqNwqhqSjVDD4RFDSq2wRgV3acbpsmw1T-NVNcClVKMPbOvURhgHQJT4UfsLA7BcgLl59qYfM7vGJ5ZNv_P8AgWs2prVSV-S_msXKU6si-3g_yfPbg6jAtKHYNgaJia1loNPqG2n4NDl3J7VoZXmKwnmACP_Bk4nlFFD3fxwxZdcGMkuj9JsRCaVA2HdCuYDUdA9wZbM5FuyA7VNpmj1-Pr2-4MZ7OhZXEEA_gKsyU1mOFTY8gROU31ZbsiFvKuhwk9VQv8Y9N-IQM2izAlKZLogtzNfe1OD8LF02YlU2VrCqdm3aNP1sUhMC0gNyBVu3qjdJbhxmJcv4fNksRx3oXIO9NO-jW1o7GgfptqluJN1I2y6l9sTK4bf1wqsMP2avRP9WhYZ6_m6HLZiTC_C2b5E_MqoRkDF9L_hG2ryrPIPOKDwUbp6ectmhKBkXq7R91etClA0FgoJNZIt79udMv1lc5BDDm5HsCEVEW1PpcbYIF2ofNsNHmWY83FExs7hNrK-SkIjht98httW3WBE_eWtYeTWm4WGZ8wGHwyhx55meY6kx5l4-8Fvw0e5YWVHw-agpEKevMXMiPZA89A_Nqt9tNIgonWg)

## Best Practices

* AWS Config Aggregators to aggregate AWS Config Data Accross all regions and accounts.
* User AWS Systems Manager to open SSH Sessions instead of unbound SSH into a jump host.
* Make sure the AWS Systems Manager Agent is running and connecting on all instances

### Check Organizational Configure
* Organizational Trail in CloudTrail, Write logs to a central account
* Capture Data Events for Lambda and S3
* Check that AWS Config is enabled for all regions and accounts
* GuardDuty Enabled for all regions and accounts


In [2]:


ransomware.check_org_trail(org_root_account, role, active_regions)
ransomware.cloudtrail_s3_events(org_root_account, active_regions, role)
ransomware.check_aws_config(active_regions, role)
ransomware.check_guardduty(active_regions, role)

### Evaluating Organization Cloudtrail

|status|Trail|Name|Account|Region|
|-----|-----|-----|-----|-----|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|Organizational Trail|383086473915|us-east-1|
|<span style="color:green"> &#9679;</span>|ENABLED|Account/Region Trail|383086473915|us-east-1|
|<span style="color:green"> &#9679;</span>|ENABLED|Account/Region Trail|383086473915|us-east-2|
|<span style="color:green"> &#9679;</span>|ENABLED|Account/Region Trail|383086473915|us-west-1|
|<span style="color:green"> &#9679;</span>|ENABLED|Account/Region Trail|383086473915|us-west-2|
|<span style="color:green"> &#9679;</span>|ENABLED|Account/Region Trail|669399753706|us-east-1|
|<span style="color:green"> &#9679;</span>|ENABLED|Account/Region Trail|669399753706|us-east-2|
|<span style="color:green"> &#9679;</span>|ENABLED|Account/Region Trail|669399753706|us-west-1|
|<span style="color:green"> &#9679;</span>|ENABLED|Account/Region Trail|669399753706|us-west-2|
|<span style="color:green"> &#9679;</span>|ENABLED|Account/Region Trail|913149361159|us-east-1|
|<span style="color:green"> &#9679;</span>|ENABLED|Account/Region Trail|913149361159|us-east-2|
|<span style="color:green"> &#9679;</span>|ENABLED|Account/Region Trail|913149361159|us-west-1|
|<span style="color:green"> &#9679;</span>|ENABLED|Account/Region Trail|913149361159|us-west-2|
|<span style="color:green"> &#9679;</span>|ENABLED|Account/Region Trail|251344881676|us-east-1|
|<span style="color:green"> &#9679;</span>|ENABLED|Account/Region Trail|251344881676|us-east-2|
|<span style="color:green"> &#9679;</span>|ENABLED|Account/Region Trail|251344881676|us-west-1|
|<span style="color:green"> &#9679;</span>|ENABLED|Account/Region Trail|251344881676|us-west-2|
|<span style="color:green"> &#9679;</span>|ENABLED|Account/Region Trail|959309154643|us-east-1|
|<span style="color:green"> &#9679;</span>|ENABLED|Account/Region Trail|959309154643|us-east-2|
|<span style="color:green"> &#9679;</span>|ENABLED|Account/Region Trail|959309154643|us-west-1|
|<span style="color:green"> &#9679;</span>|ENABLED|Account/Region Trail|959309154643|us-west-2|


### Evaluating Cloudtrail S3 Data Events

|status|S3 Data Events|Account|Region|
|-----|-----|-----|-----|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|383086473915|us-east-1|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|383086473915|us-east-2|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|383086473915|us-west-1|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|383086473915|us-west-2|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|669399753706|us-east-1|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|669399753706|us-east-2|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|669399753706|us-west-1|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|669399753706|us-west-2|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|913149361159|us-east-1|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|913149361159|us-east-2|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|913149361159|us-west-1|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|913149361159|us-west-2|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|251344881676|us-east-1|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|251344881676|us-east-2|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|251344881676|us-west-1|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|251344881676|us-west-2|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|959309154643|us-east-1|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|959309154643|us-east-2|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|959309154643|us-west-1|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|959309154643|us-west-2|


### Evaluating AWS Config Recorders

|status|Config Recorder|Account|Region|
|-----|-----|-----|-----|
|<span style="color:green"> &#9679;</span>|ENABLED|383086473915|us-east-1|
|<span style="color:green"> &#9679;</span>|ENABLED|383086473915|us-east-2|
|<span style="color:green"> &#9679;</span>|ENABLED|383086473915|us-west-1|
|<span style="color:green"> &#9679;</span>|ENABLED|383086473915|us-west-2|
|<span style="color:red"> &#9679;</span>|DISABLED|669399753706|us-east-1|
|<span style="color:red"> &#9679;</span>|DISABLED|669399753706|us-east-2|
|<span style="color:red"> &#9679;</span>|DISABLED|669399753706|us-west-1|
|<span style="color:red"> &#9679;</span>|DISABLED|669399753706|us-west-2|
|<span style="color:green"> &#9679;</span>|ENABLED|913149361159|us-east-1|
|<span style="color:green"> &#9679;</span>|ENABLED|913149361159|us-east-2|
|<span style="color:green"> &#9679;</span>|ENABLED|913149361159|us-west-1|
|<span style="color:green"> &#9679;</span>|ENABLED|913149361159|us-west-2|
|<span style="color:green"> &#9679;</span>|ENABLED|251344881676|us-east-1|
|<span style="color:green"> &#9679;</span>|ENABLED|251344881676|us-east-2|
|<span style="color:green"> &#9679;</span>|ENABLED|251344881676|us-west-1|
|<span style="color:green"> &#9679;</span>|ENABLED|251344881676|us-west-2|
|<span style="color:red"> &#9679;</span>|DISABLED|959309154643|us-east-1|
|<span style="color:red"> &#9679;</span>|DISABLED|959309154643|us-east-2|
|<span style="color:red"> &#9679;</span>|DISABLED|959309154643|us-west-1|
|<span style="color:red"> &#9679;</span>|DISABLED|959309154643|us-west-2|


### Evaluating AWS GuardDuty

|status|GuardDuty|Account|Region|
|-----|-----|-----|-----|
|<span style="color:green"> &#9679;</span>|ENABLED|383086473915|us-east-1|
|<span style="color:green"> &#9679;</span>|ENABLED|383086473915|us-east-2|
|<span style="color:green"> &#9679;</span>|ENABLED|383086473915|us-west-1|
|<span style="color:green"> &#9679;</span>|ENABLED|383086473915|us-west-2|
|<span style="color:green"> &#9679;</span>|ENABLED|669399753706|us-east-1|
|<span style="color:green"> &#9679;</span>|ENABLED|669399753706|us-east-2|
|<span style="color:green"> &#9679;</span>|ENABLED|669399753706|us-west-1|
|<span style="color:green"> &#9679;</span>|ENABLED|669399753706|us-west-2|
|<span style="color:green"> &#9679;</span>|ENABLED|913149361159|us-east-1|
|<span style="color:green"> &#9679;</span>|ENABLED|913149361159|us-east-2|
|<span style="color:green"> &#9679;</span>|ENABLED|913149361159|us-west-1|
|<span style="color:green"> &#9679;</span>|ENABLED|913149361159|us-west-2|
|<span style="color:green"> &#9679;</span>|ENABLED|251344881676|us-east-1|
|<span style="color:green"> &#9679;</span>|ENABLED|251344881676|us-east-2|
|<span style="color:green"> &#9679;</span>|ENABLED|251344881676|us-west-1|
|<span style="color:green"> &#9679;</span>|ENABLED|251344881676|us-west-2|
|<span style="color:green"> &#9679;</span>|ENABLED|959309154643|us-east-1|
|<span style="color:green"> &#9679;</span>|ENABLED|959309154643|us-east-2|
|<span style="color:green"> &#9679;</span>|ENABLED|959309154643|us-west-1|
|<span style="color:green"> &#9679;</span>|ENABLED|959309154643|us-west-2|


## Validate Inspector

Check each region to see if inspector is active.



In [3]:
import importlib
importlib.reload(ransomware)


ransomware.cloudtrail_s3_events(org_root_account, active_regions, role)

### Evaluating Cloudtrail S3 Data Events

|status|S3 Data Events|Account|Region|
|-----|-----|-----|-----|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|383086473915|us-east-1|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|383086473915|us-east-2|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|383086473915|us-west-1|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|383086473915|us-west-2|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|669399753706|us-east-1|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|669399753706|us-east-2|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|669399753706|us-west-1|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|669399753706|us-west-2|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|913149361159|us-east-1|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|913149361159|us-east-2|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|913149361159|us-west-1|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|913149361159|us-west-2|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|251344881676|us-east-1|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|251344881676|us-east-2|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|251344881676|us-west-1|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|251344881676|us-west-2|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|959309154643|us-east-1|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|959309154643|us-east-2|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|959309154643|us-west-1|
|<span style="color:red"> &#9679;</span>|NOT ENABLED|959309154643|us-west-2|


# Instance Management
1. Based on AWS SSM agent running on the intances
1. Vulnerbility Management by Inspector
1. Validate patch status via patch manager.

## Patch Management

The next step is to view the current patch status of the EC2 instances

In [4]:
import importlib
importlib.reload(ransomware)

ransomware.check_patch_manager(active_regions, role)


### Evaluating SSM Patch Manager activation

|status|Instance ID|Patch Group|CriticalNonCompliantCount|Account|Region|
|-----|-----|-----|-----|-----|-----|
|<span style="color:green"> &#9679;</span>|i-00ef2bba7ff72e968|AccountGuardian-PatchGroup-DO-NOT-DELETE|0|383086473915|us-east-1|
|<span style="color:green"> &#9679;</span>|i-0a085da0e9349b2f1|AccountGuardian-PatchGroup-DO-NOT-DELETE|0|383086473915|us-east-1|
|<span style="color:green"> &#9679;</span>|i-0b191121f743031ec|AccountGuardian-PatchGroup-DO-NOT-DELETE|0|383086473915|us-east-1|
|<span style="color:green"> &#9679;</span>|i-0b6b68ea2f2ba98c9|AccountGuardian-PatchGroup-DO-NOT-DELETE|0|383086473915|us-east-1|
|<span style="color:green"> &#9679;</span>|i-022c09a8d6fb085a3||0|669399753706|us-west-2|
|<span style="color:green"> &#9679;</span>|i-08af9e6b33d7c7fd4||0|251344881676|us-east-2|


## Check for public facing instances
The following step will use AWS Config to list any EC2 instances that have a public IP address.


In [5]:
import importlib
importlib.reload(ransomware)
ransomware.check_public_ips(configuration_aggregator)

### Public IPs

|status|accountId|awsRegion|instanceId|publicIp|
|-----|-----|-----|-----|-----|
|<span style="color:green"> &#9679;</span>|383086473915|us-east-1|i-0a085da0e9349b2f1|18.208.228.76|
|<span style="color:green"> &#9679;</span>|251344881676|us-east-2|i-08af9e6b33d7c7fd4|3.16.137.113|
|<span style="color:green"> &#9679;</span>|383086473915|us-east-1|i-0b6b68ea2f2ba98c9|100.25.37.242|
|<span style="color:green"> &#9679;</span>|383086473915|us-east-1|i-0b191121f743031ec|44.204.152.218|


## DNS Firewall
TO DO: How do we check for a DNS firewall?

## Check for SSH unrestricted SSH Inbound
Strongly encouraged to block inbound SSH for the CIDR range 0.0.0.0. This will check the config rule for Organization-Wide SSH access.

If you currently allow SSH, consider using AWS Systems Manage Session Manager.

In [6]:
ransomware.check_restricted_ssh_config_rule(configuration_aggregator, role, active_regions)

### Restricted SSHs

|status|ResourceType|ResourceId|ComplianceType|AccountId|AwsRegion|
|-----|-----|-----|-----|-----|-----|
|<span style="color:green"> &#9679;</span>|organization_config_rule|org-restricted-ssh|COMPLIANT|&nbsp;|&nbsp;|
|<span style="color:red"> &#9679;</span>|AWS::EC2::SecurityGroup|sg-0d011ac00f029463e|NON_COMPLIANT|383086473915|us-east-1|
|<span style="color:red"> &#9679;</span>|AWS::EC2::SecurityGroup|sg-a39816de|NON_COMPLIANT|383086473915|us-east-1|
|<span style="color:red"> &#9679;</span>|AWS::EC2::SecurityGroup|sg-0a40fd48e3e0a6065|NON_COMPLIANT|913149361159|us-east-1|
|<span style="color:red"> &#9679;</span>|AWS::EC2::SecurityGroup|sg-004b807a4daa3ef73|NON_COMPLIANT|251344881676|us-east-1|
|<span style="color:red"> &#9679;</span>|AWS::EC2::SecurityGroup|sg-0d2b6d4e44abeba98|NON_COMPLIANT|251344881676|us-east-1|
|<span style="color:red"> &#9679;</span>|AWS::EC2::SecurityGroup|sg-0d30a8994cdbb8a0e|NON_COMPLIANT|251344881676|us-east-1|


In [7]:
ransomware.save_report(destination_bucket, destination_prefix)